In [28]:
# Loading all the libraries
import pandas as pd
import numpy as np
import scipy.stats
import streamlit as st
import time
import plotly.express as px


In [29]:
#Load Data

df = pd.read_csv('../vehicles_us.csv')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [31]:
print ("info: \n",df['type'].info())

print("head: \n",df['type'].head(n=15))

print("sample: \n",df['type'].sample(n=15))

<class 'pandas.core.series.Series'>
RangeIndex: 51525 entries, 0 to 51524
Series name: type
Non-Null Count  Dtype 
--------------  ----- 
51525 non-null  object
dtypes: object(1)
memory usage: 402.7+ KB
info: 
 None
head: 
 0        SUV
1     pickup
2      sedan
3     pickup
4      sedan
5      sedan
6      sedan
7        SUV
8        SUV
9        SUV
10    pickup
11     sedan
12    pickup
13     sedan
14       SUV
Name: type, dtype: object
sample: 
 11993       SUV
18804       SUV
27724     truck
13757       SUV
16217     sedan
32688       SUV
790       truck
20179       SUV
32018    pickup
49379     truck
14438    pickup
33535     truck
25061     sedan
14945       SUV
29887     wagon
Name: type, dtype: object


In [32]:
def explore_df(df, name):
    print("="*50)
    print(f"Exploring Column: \"{name}\"")
    print("First few rows:")
    print(df.head(), "\n")
    print("Data types:")
    print(df.dtypes, "\n")
    print("Summary statistics:")
    print(df.describe(include='all'), "\n")
    print("Data Frame Info:")
    print(df.info(), "\n")
    print("Get 20 random rows:")
    print(df.sample(n=20), "\n")
    print("="*50, "\n")

df.info()

for col in df:
    explore_df(df[col],col)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB
Exploring Column: "price"
First few rows:
0     9400
1    25500
2     5500
3     1500
4    14900
Name: price, dtype: int64 

Data types:
int64 

Summary statistics:
count     51525.000

# Correct datatypes

0   price         51525 non-null  int64    #Should be float?

1   model_year    47906 non-null  float64  #Should be int, missing objects, but can they be replaced?

2   model         51525 non-null  object 

3   condition     51525 non-null  object   

4   cylinders     46265 non-null  float64  #Should be int, missing data

5   fuel          51525 non-null  object 

6   odometer      43633 non-null  float64  #missing data

7   transmission  51525 non-null  object

8   type          51525 non-null  object 

9   paint_color   42258 non-null  object   #missing data

10  is_4wd        25572 non-null  float64  #Should be boolean, not binary, not float

11  date_posted   51525 non-null  object   #Should be datetype

12  days_listed   51525 non-null  int64 

# Drop or augment missing values:

1. drop rows with missing model_year. No way they can be replaced without significantly skewing data.
2. cylinders, calculate the most common value for the model vehicle and assign, if this fails, assign most common in all data
3. odometer, calcutate mean value according to condition
4. paint_color, calculate most common value for model, else assiggn most common value in all data
5. is_4wd defaults to false if 0.0 in dataframe.

In [33]:
def dataprep(df):
    """
    Cleans and converts columns in the DataFrame to the desired data types and drops rows with missing data.

    Parameters:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    pd.DataFrame: The cleaned and converted DataFrame.
    """
    # Drop rows with missing model_year
    df = df.dropna(subset=['model_year'])

    # Fill missing cylinders values
    # Calculate the mean value for each model
    model_cylinders_mean = df.groupby('model')['cylinders'].mean()
    df['cylinders'] = df.apply(
        lambda row: model_cylinders_mean[row['model']] if pd.isna(row['cylinders']) else row['cylinders'], axis=1
    )
    # Fill any remaining NaN values with the overall mean cylinders value
    overall_mean_cylinders = df['cylinders'].mean()
    df['cylinders'] = df['cylinders'].fillna(overall_mean_cylinders)

    # Fill missing odometer values
    # Calculate the mean odometer value for each condition
    condition_odometer = df.groupby('condition')['odometer'].mean()
    df['odometer'] = df.apply(
        lambda row: condition_odometer[row['condition']] if pd.isna(row['odometer']) else row['odometer'], axis=1
    )

    # Fill missing paint_color values
    # Calculate the most common value for each model
    model_paint_color = df.groupby('model')['paint_color'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    df['paint_color'] = df.apply(
        lambda row: model_paint_color[row['model']] if pd.isna(row['paint_color']) else row['paint_color'], axis=1
    )
    # Fill any remaining NaN values with the overall most common paint color
    most_common_paint_color = df['paint_color'].mode().iloc[0]
    df['paint_color'] = df['paint_color'].fillna(most_common_paint_color)

    # Fill missing is_4wd values with 0.0
    df['is_4wd'] = df['is_4wd'].fillna(0.0)

    # Convert data types
    df['price'] = df['price'].astype('float64')
    df['model_year'] = df['model_year'].astype('int64')
    df['cylinders'] = df['cylinders'].astype('int64')
    df['is_4wd'] = df['is_4wd'].map({1.0: True, 0.0: False})
    df['date_posted'] = pd.to_datetime(df['date_posted']).dt.floor('D')

    return df

df_clean = dataprep(df)

df_clean.info()
print("First few rows:")
print(df_clean.head(n=50), "\n")

# What do we see?
I'm imagining the type of company similar to CoPart. I twould be useful to make it modular so the user is able to perform simple data visualization by taking in filters.

- What is the most popular car?

- Market trends ? -  What is the average car price of all years?
    

In [ ]:
def plot_histogram_price_filtered(df, model_year=None, cylinders=None, condition=None, fuel=None, transmission=None, 
                                  car_type=None, paint_color=None, is_4wd=None, models=None, aggregation='Average Vehicle Price'):
    """
    Plots a histogram of car prices by model with optional filters.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    model_year (list or int, optional): The specific model year(s) to filter and plot. Defaults to None.
    cylinders (list or int, optional): The number of cylinders to filter and plot. Defaults to None.
    condition (list or str, optional): The condition of the car to filter and plot. Defaults to None.
    fuel (list or str, optional): The type of fuel to filter and plot. Defaults to None.
    transmission (list or str, optional): The type of transmission to filter and plot. Defaults to None.
    car_type (list or str, optional): The type of car to filter and plot. Defaults to None.
    paint_color (list or str, optional): The paint color of the car to filter and plot. Defaults to None.
    is_4wd (list or bool, optional): Whether the car is 4WD to filter and plot. Defaults to None.
    models (list of str, optional): List of substrings to filter models. Defaults to None.
    aggregation (str, optional): The aggregation method for price ('Average Price' or 'Market Capitalization'). Defaults to 'Average Price'.
    """
    # Apply filters
    filters_applied = {
        'model_year': model_year,
        'cylinders': cylinders,
        'condition': condition,
        'fuel': fuel,
        'transmission': transmission,
        'car_type': car_type,
        'paint_color': paint_color,
        'is_4wd': is_4wd,
        'models': models
    }

    # Treat empty lists as None
    for key, value in filters_applied.items():
        if isinstance(value, list) and len(value) == 0:
            filters_applied[key] = None

    if filters_applied['model_year'] is not None:
        if isinstance(filters_applied['model_year'], list):
            df = df[df['model_year'].isin(filters_applied['model_year'])]
        else:
            df = df[df['model_year'] == filters_applied['model_year']]
    if filters_applied['cylinders'] is not None:
        if isinstance(filters_applied['cylinders'], list):
            df = df[df['cylinders'].isin(filters_applied['cylinders'])]
        else:
            df = df[df['cylinders'] == filters_applied['cylinders']]
    if filters_applied['condition'] is not None:
        if isinstance(filters_applied['condition'], list):
            df = df[df['condition'].isin(filters_applied['condition'])]
        else:
            df = df[df['condition'] == filters_applied['condition']]
    if filters_applied['fuel'] is not None:
        if isinstance(filters_applied['fuel'], list):
            df = df[df['fuel'].isin(filters_applied['fuel'])]
        else:
            df = df[df['fuel'] == filters_applied['fuel']]
    if filters_applied['transmission'] is not None:
        if isinstance(filters_applied['transmission'], list):
            df = df[df['transmission'].isin(filters_applied['transmission'])]
        else:
            df = df[df['transmission'] == filters_applied['transmission']]
    if filters_applied['car_type'] is not None:
        if isinstance(filters_applied['car_type'], list):
            df = df[df['type'].isin(filters_applied['car_type'])]
        else:
            df = df[df['type'] == filters_applied['car_type']]
    if filters_applied['paint_color'] is not None:
        if isinstance(filters_applied['paint_color'], list):
            df = df[df['paint_color'].isin(filters_applied['paint_color'])]
        else:
            df = df[df['paint_color'] == filters_applied['paint_color']]
    if filters_applied['is_4wd'] is not None:
        if isinstance(filters_applied['is_4wd'], list):
            df = df[df['is_4wd'].isin(filters_applied['is_4wd'])]
        else:
            df = df[df['is_4wd'] == filters_applied['is_4wd']]
    if filters_applied['models'] is not None:
        model_pattern = '|'.join(filters_applied['models'])
        df = df[df['model'].str.contains(model_pattern, case=False, na=False)]
    
    # Set the aggregation function for price
    if aggregation == 'Average Vehicle Price':
        df_grouped = df.groupby('model')['price'].mean().reset_index()
        y_title = 'Average Vehicle Price'
    else:
        df_grouped = df.groupby('model')['price'].sum().reset_index()
        y_title = 'Market Capitalization'
    
    # Create the bar chart
    fig = px.bar(df_grouped, x='model', y='price',
                 title="Histogram of Car Prices by Model" + (f" for {filters_applied['model_year']}" if filters_applied['model_year'] else ""),
                 color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_layout(xaxis_title='Model', yaxis_title=y_title, template='plotly_white')

    return fig, len(df)



#Scatterplot

def plot_scatterplot_price_year(df, model_year=None, cylinders=None, condition=None, fuel=None, transmission=None, 
                                car_type=None, paint_color=None, is_4wd=None, models=None, aggregation="Average Vehicle Price"):
    """
    Plots a scatterplot of car prices over the years with optional filters.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    model_year (list or int, optional): The specific model year(s) to filter and plot. Defaults to None.
    cylinders (list or int, optional): The number of cylinders to filter and plot. Defaults to None.
    condition (list or str, optional): The condition of the car to filter and plot. Defaults to None.
    fuel (list or str, optional): The type of fuel to filter and plot. Defaults to None.
    transmission (list or str, optional): The type of transmission to filter and plot. Defaults to None.
    car_type (list or str, optional): The type of car to filter and plot. Defaults to None.
    paint_color (list or str, optional): The paint color of the car to filter and plot. Defaults to None.
    is_4wd (list or bool, optional): Whether the car is 4WD to filter and plot. Defaults to None.
    models (list of str, optional): List of substrings to filter models. Defaults to None.
    aggregation (str, optional): The aggregation method for price ('Average Price' or 'Market Capitalization'). Defaults to 'Average Price'.
    """
    # Apply filters
    filters_applied = {
        'model_year': model_year,
        'cylinders': cylinders,
        'condition': condition,
        'fuel': fuel,
        'transmission': transmission,
        'car_type': car_type,
        'paint_color': paint_color,
        'is_4wd': is_4wd,
        'models': models
    }

    # Treat empty lists as None
    for key, value in filters_applied.items():
        if isinstance(value, list) and len(value) == 0:
            filters_applied[key] = None

    if filters_applied['model_year'] is not None:
        if isinstance(filters_applied['model_year'], list):
            df = df[df['model_year'].isin(filters_applied['model_year'])]
        else:
            df = df[df['model_year'] == filters_applied['model_year']]
    if filters_applied['cylinders'] is not None:
        if isinstance(filters_applied['cylinders'], list):
            df = df[df['cylinders'].isin(filters_applied['cylinders'])]
        else:
            df = df[df['cylinders'] == filters_applied['cylinders']]
    if filters_applied['condition'] is not None:
        if isinstance(filters_applied['condition'], list):
            df = df[df['condition'].isin(filters_applied['condition'])]
        else:
            df = df[df['condition'] == filters_applied['condition']]
    if filters_applied['fuel'] is not None:
        if isinstance(filters_applied['fuel'], list):
            df = df[df['fuel'].isin(filters_applied['fuel'])]
        else:
            df = df[df['fuel'] == filters_applied['fuel']]
    if filters_applied['transmission'] is not None:
        if isinstance(filters_applied['transmission'], list):
            df = df[df['transmission'].isin(filters_applied['transmission'])]
        else:
            df = df[df['transmission'] == filters_applied['transmission']]
    if filters_applied['car_type'] is not None:
        if isinstance(filters_applied['car_type'], list):
            df = df[df['type'].isin(filters_applied['car_type'])]
        else:
            df = df[df['type'] == filters_applied['car_type']]
    if filters_applied['paint_color'] is not None:
        if isinstance(filters_applied['paint_color'], list):
            df = df[df['paint_color'].isin(filters_applied['paint_color'])]
        else:
            df = df[df['paint_color'] == filters_applied['paint_color']]
    if filters_applied['is_4wd'] is not None:
        if isinstance(filters_applied['is_4wd'], list):
            df = df[df['is_4wd'].isin(filters_applied['is_4wd'])]
        else:
            df = df[df['is_4wd'] == filters_applied['is_4wd']]
    if filters_applied['models'] is not None:
        model_pattern = '|'.join(filters_applied['models'])
        df = df[df['model'].str.contains(model_pattern, case=False, na=False)]
    
    # Set the aggregation function for price
    if aggregation == "Average Vehicle Price":
        df_grouped = df.groupby('model_year')['price'].mean().reset_index()
        y_title = 'Average Vehicle Price'
    else:
        df_grouped = df.groupby('model_year')['price'].sum().reset_index()
        y_title = 'Market Capitalization'
    
    # Create the scatter plot
    fig = px.scatter(df_grouped, x='model_year', y='price', 
                     title="Scatterplot of Car Prices over Years",
                     labels={'model_year': 'Model Year', 'price': y_title},
                     color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_layout(xaxis_title='Model Year', yaxis_title=y_title, template='plotly_white')

    return fig, len(df)


# Plot histogram with no filterd and aggregation as Market Capitalizaiton (sum)
fig_histo, count_histo = plot_histogram_price_filtered(df_clean, aggregation='Market Capitalization')
fig_histo.show()
print("Vehicles in plot: ",count_histo)

# Plot scatterplot for all model years with specific filters and default aggregation (mean)
fig_scatter_mean, count_scatter_mean = plot_scatterplot_price_year(df_clean, aggregation='Average Vehicle Price')
fig_scatter_mean.show()
print("Vehicles in plot: ",count_scatter_mean)

# Plot scatterplot for all model years with specific filters and Market Capitalization  (sum)
fig_scatter_sum, count_scatter_sum = plot_scatterplot_price_year(df_clean, aggregation='Market Capitalization')
fig_scatter_sum.show()
print("Vehicles in plot: ",count_scatter_sum)

Vehicles in plot:  47906


Vehicles in plot:  47906


Vehicles in plot:  47906


# Conclusions

1. The most popular vehicles are trucks: 
    1. F150
    2. Chevrolet Silverado 
    4. RAM

2. Vehicles have their lowest value in the 2000s, current year is mid 2019, so this means lowest car values hit during late teens.

3. Most frequent model year is 2015, since data is from mod 2019 it means that there4 are more 4 year old cars on the platform. Perhaps it's normal to get a new car every 4 years? Do Americas really do this??? I might need a new car .....